In [1]:
import pandas as pd
import json

In [2]:
def read_json_file(path: str) -> dict:
    with open(path, 'r') as f:
        return json.load(f)

In [20]:
path = '../data/mpd.slice.0-999.json'
data = read_json_file(path)

In [22]:
df_musicas = pd.DataFrame()

# Itera sobre as playlists
for playlist in data['playlists']:
    name_playlist = playlist['name']
    id_playlist = playlist['pid']

    print(f'{id_playlist} - Playlist: {name_playlist}')

    # Itera sobre as músicas da playlist
    musicas_playlist = playlist['tracks']

    for musica in musicas_playlist:
        name_musica = musica['track_name']
        id_musica = musica['track_uri']

        # print(f'\tMúsica: {name_musica} - ID: {id_musica}')
        df_musica_aux = pd.DataFrame.from_dict(musica, orient='index')
        df_musica_aux = df_musica_aux.T

        # Adiciona o nome da playlist e o id da playlist


        df_musica_aux['playlist_name'] = name_playlist
        df_musica_aux['playlist_id'] = id_playlist

        df_musica_aux = df_musica_aux.T
        df_musicas = pd.concat([df_musicas, df_musica_aux], axis=1)
        
        # break
    break

df_musicas = df_musicas.T
# df_musicas.to_csv('data/df_musicas_parte_1.csv', index=False)

0 - Playlist: Throwbacks


In [23]:
df_musicas = df_musicas[['track_uri', 'artist_uri', 'album_uri', 'track_name', 'playlist_name', 'playlist_id', 'album_name', 'duration_ms']]

df_musicas['track_uri'] = df_musicas['track_uri'].str.replace('spotify:track:', '')
df_musicas['artist_uri'] = df_musicas['artist_uri'].str.replace('spotify:artist:', '')
df_musicas['album_uri'] = df_musicas['album_uri'].str.replace('spotify:album:', '')

df_musicas

,track_uri,artist_uri,album_uri,track_name,playlist_name,playlist_id,album_name,duration_ms
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K,Lose Control (feat. Ciara & Fat Man Scoop),Throwbacks,0,The Cookbook,226863
0,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI,Toxic,Throwbacks,0,In The Zone,198800
0,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE,Crazy In Love,Throwbacks,0,Dangerously In Love (Alben für die Ewigkeit),235933
0,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW,Rock Your Body,Throwbacks,0,Justified,267266
0,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF,It Wasn't Me,Throwbacks,0,Hot Shot,227600
0,0XUfyU2QviPAs6bxSpXYG4,23zg3TcAtWQy7J6upgbUnj,0vO0b1AvY49CPQyVisJLj0,Yeah!,Throwbacks,0,Confessions,250373
0,68vgtRHr7iZHpzGpon6Jlo,23zg3TcAtWQy7J6upgbUnj,1RM6MGv6bcl6NrAG8PGoZk,My Boo,Throwbacks,0,Confessions,223440
0,3BxWKCI06eQ5Od8TY2JBeA,6wPhSqRtPu1UhRCDX5yaDJ,5x8e8UcCeOgrOzSnDGuPye,Buttons,Throwbacks,0,PCD,225560
0,7H6ev70Weq6DdpZyyTmUXk,1Y8cdNmUJH7yBTd9yOvr5i,283NWqNsCA9GwVHrJk59CG,Say My Name,Throwbacks,0,The Writing's On The Wall,271333
0,2PpruBYCo4H7WOBJ7Q2EwM,1G9G7WwrXka3Z1r7aIDjI7,1UsmQ3bpJTyK6ygoOOjG1r,Hey Ya! - Radio Mix / Club Mix,Throwbacks,0,Speakerboxxx/The Love Below,235213


In [8]:
# Músicas da playlist
data['playlists'][0]['tracks'][0]

{'pos': 0,
 'artist_name': 'Missy Elliott',
 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk',
 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K',
 'duration_ms': 226863,
 'album_name': 'The Cookbook'}